In [781]:
pwd

'/Users/alexrosheim/Desktop/UDENDATA_ACR/04-Pandas Homework_ACR/HeroesOfPymoli'

In [782]:
import pandas as pd
import numpy as np

In [783]:
csv_file = "Resources/purchase_data.csv"
df = pd.read_csv(csv_file)

In [784]:
# Purchasing Analysis (Total)

In [785]:
total_players = df['SN'].unique().size
df['Total Players'] = total_players

uniq_items = df['Item ID'].unique().size
df['Num of Unique Items'] = uniq_items
df['Num of Unique Items'] = df['Num of Unique Items'].map(str)

avg_purch = df['Price'].mean()
df['Avg Price'] = "${0:.2f}".format(avg_purch) 

total_purch = df['Purchase ID'].count()
df['Num of Purchases'] = total_purch

total_rev = df['Price'].sum()
df['Total Revenue'] = "${0:.2f}".format(total_rev)

purch_analysis = df[['Total Players', 'Num of Unique Items', 'Avg Price', 'Num of Purchases', 'Total Revenue']]
purch_analysis.head(1)

,Total Players,Num of Unique Items,Avg Price,Num of Purchases,Total Revenue
0,576,183,$3.05,780,$2379.77


In [786]:
# Gender Demographics

In [787]:
gen_total = df.groupby(['Gender','SN']).count().reset_index()['Gender'].value_counts(normalize=False).rename('Gender_Count')
gen_percent = df.groupby(['Gender','SN']).count().reset_index()['Gender'].value_counts(normalize=True).map(('{:.2%}'.format)).rename('Gender_Pct')
gen_demo = pd.concat([gen_percent,gen_total], axis=1)
gen_demo

,Gender_Pct,Gender_Count
Male,84.03%,484
Female,14.06%,81
Other / Non-Disclosed,1.91%,11


In [788]:
# Purchasing Analysis (Gender)

In [789]:
g_demo = df.groupby('Gender').agg(['sum', 'mean', 'count'])['Price']
g_demo = pd.concat([g_demo, gen_total], axis=1, sort=True)
g_demo['Avg Total per Person'] = g_demo['sum'] / g_demo['Gender_Count']
g_demo = g_demo.rename(columns={'sum':'Total Purchase Value', 'mean':'Avg Purchase Price', 'count': 'Purchase Count'})
g_demo['Total Purchase Value'] = g_demo['Total Purchase Value'].map('${0:.2f}'.format)
g_demo['Avg Purchase Price'] = g_demo['Avg Purchase Price'].map('${0:.2f}'.format)
g_demo['Avg Total per Person'] = g_demo['Avg Total per Person'].map('${0:.2f}'.format)
g_demo

,Total Purchase Value,Avg Purchase Price,Purchase Count,Gender_Count,Avg Total per Person
Female,$361.94,$3.20,113,81,$4.47
Male,$1967.64,$3.02,652,484,$4.07
Other / Non-Disclosed,$50.19,$3.35,15,11,$4.56


In [790]:
# Age Demographics

In [791]:
bins = [0, 14, 18, 21, 25, 30, 35, 40, 50]
group_names = ["0-14 yrs", "15-18 yrs", "19-21 yrs", "22-25 yrs", "26-30 yrs", "31-35 yrs", "36-40 yrs", "41-50 yrs"]
df["Age_Bins"] = pd.cut(df["Age"], bins, labels=group_names)
age_percent = df['Age_Bins'].value_counts(normalize=True).map('{:.2%}'.format).rename('Age_Pct')
age_total = df['Age_Bins'].value_counts(normalize=False).rename('Age_Count')
age_demo = pd.concat([age_percent, age_total], axis=1)
age_demo.index.name = 'Age Bins'
age_demo

,Age_Pct,Age_Count
Age Bins,,
22-25 yrs,33.72%,263
19-21 yrs,23.59%,184
15-18 yrs,14.49%,113
26-30 yrs,9.87%,77
31-35 yrs,6.67%,52
0-14 yrs,6.54%,51
36-40 yrs,4.23%,33
41-50 yrs,0.90%,7


In [792]:
# Age Demographics (continued)

In [793]:
age_demo['Purchase Count'] = df.groupby('Age_Bins')['Price'].agg(['count'])
age_demo['Avg Purchase Price'] = df.groupby('Age_Bins')['Price'].agg(['mean'])
age_demo['Total Purchase Value'] = df.groupby('Age_Bins')['Price'].agg(['sum'])
age_demo['Avg Purchase Price'] = age_demo['Avg Purchase Price'].map('${0:.2f}'.format)
age_demo['Total Purchase Value'] = age_demo['Total Purchase Value'].map('${0:.2f}'.format)
age_demo['Avg Purchase per Person'] = df.drop_duplicates('SN').groupby('Age_Bins')['Price'].agg(['mean'])
age_demo['Avg Purchase per Person'] = age_demo['Avg Purchase per Person'].map('${0:.2f}'.format)
age_demo = age_demo[['Purchase Count', 'Avg Purchase Price', 'Total Purchase Value', 'Avg Purchase per Person']]
age_demo


,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Purchase per Person
Age Bins,,,,
22-25 yrs,263,$3.05,$800.90,$3.04
19-21 yrs,184,$3.07,$565.04,$3.13
15-18 yrs,113,$3.03,$342.91,$3.08
26-30 yrs,77,$2.88,$221.42,$2.90
31-35 yrs,52,$2.99,$155.71,$3.00
0-14 yrs,51,$3.14,$159.91,$3.21
36-40 yrs,33,$3.40,$112.35,$3.29
41-50 yrs,7,$3.08,$21.53,$3.08


In [794]:
# Top Spenders

In [795]:
spenders = df.groupby('SN')['Price'].agg(['sum', 'mean', 'count'])
spenders = spenders.rename(columns={'sum':'Total Purchase Value', 'mean':'Item Price', 'count': 'Purchase Count'})
spenders['Item Price'] = spenders['Item Price'].map('${0:.2f}'.format)
spenders = spenders.sort_values(by='Total Purchase Value', ascending=False)
spenders['Total Purchase Value'] = spenders['Total Purchase Value'].map('${0:.2f}'.format)
spenders.head()

,Total Purchase Value,Item Price,Purchase Count
SN,,,
Lisosia93,$18.96,$3.79,5
Idastidru52,$15.45,$3.86,4
Chamjask73,$13.83,$4.61,3
Iral74,$13.62,$3.40,4
Iskadarya95,$13.10,$4.37,3


In [796]:
# Most Popular Items

In [797]:
popular = df.groupby(['Item ID','Item Name'])['Price'].agg(['count', 'mean', 'sum'])
popular = popular.rename(columns={'sum':'Total Purchase Value', 'mean':'Item Price', 'count': 'Purchase Count'})
popular['Item Price'] = popular['Item Price'].map('${0:.2f}'.format)
popular['Total Purchase Value'] = popular['Total Purchase Value'].map('${0:.2f}'.format)
popular.sort_values(by='Purchase Count', ascending=False).head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [798]:
# Most Profitable Items

In [799]:
profitable = df.groupby(['Item ID','Item Name'])['Price'].agg(['count', 'mean', 'sum'])
profitable = profitable.rename(columns={'sum':'Total Purchase Value', 'mean':'Item Price', 'count': 'Purchase Count'})
profitable['Item Price'] = profitable['Item Price'].map('${0:.2f}'.format)
profitable = profitable.sort_values(by='Total Purchase Value', ascending=False)
profitable['Total Purchase Value'] = profitable['Total Purchase Value'].map('${0:.2f}'.format)
profitable.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


In [800]:
# You must include a written description of three observable trends based on the data.

#Age Demographics

Over 1/3 of our users fall into the age demographic of 22-25 yrs, with 263 of the total 576 users.
The next highest age demographics are 19-21 yrs, with 184 users, and 15-18 yrs with 113 users.
Totgether, these three age demographics account for 71.8% of all users.

In [802]:
repeat_purchase = gen_demo['Gender_Count'] / df.groupby('Gender').count()['Price']
repeat_purchase = (1 - repeat_purchase).map('{:.2%}'.format)
repeat_purchase

Female                   28.32%
Male                     25.77%
Other / Non-Disclosed    26.67%
dtype: object

#Gender Demographics

As anticipated, there are considerably more men than women who are active players of our games.
Of the 576 total users, 84% are male, with 484 users, and 14% are female with 81 users.

Additinally, many of these users purchase more than one of our games.
We can see a trend that over 25% of our total users will also purchase more than game.
Female users lead that category with 28% of their users purchasing additional games.
Their Male counter parts are a litte less likely to purchase additinal games, but are still over the trend of 25% being repeat customers.